indie's section

In [14]:
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)
players <- read_csv("Data/players.csv")
sessions <- read_csv("Data/sessions.csv")

sessions_cleaned <- sessions |> mutate(email = as_factor(hashedEmail)) |> select(-hashedEmail)

players_sorted <- players |> mutate(email = as_factor(hashedEmail))
sessions_named <- sessions_cleaned |> left_join(players_sorted, by = "email") |> select(-hashedEmail)

sessions_final <- sessions_named |> mutate(start = dmy_hm(start_time), 
end = dmy_hm(end_time), start_date = as_date(start), start_time = format(start, "%H:%M"), 
end_date = as_date(end_time), end_time = format(end, "%H:%M")) |>
select(-start, -end, -email, -start_date, -end_date)
sessions <- sessions_final |>  mutate(start = as.POSIXct(paste("2025-01-01", start_time), format = "%Y-%m-%d %H:%M"),
end = as.POSIXct(paste("2025-01-01", end_time), format = "%Y-%m-%d %H:%M")) |> mutate(end = if_else(end < start, end + lubridate::days(1), end)) |> 
mutate(session_minutes = as.numeric(difftime(end, start, units = "mins"))) |> select(-start, -end, -original_start_time, -original_end_time) |>
group_by(name, subscribe) |> summarize(average_session = mean(session_minutes))
sessions


Rows: 196 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, Age
lgl (1): subscribe

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 1535 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): hashedEmail, start_time, end_time
dbl (2): original_start_time, original_end_time

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
`summarise()` has grouped output by 'name'. You can override using the
`.groups` argument.


name,subscribe,average_session
<chr>,<lgl>,<dbl>
Aarav,TRUE,18
Aaron,TRUE,43
Ahmed,TRUE,15
⋮,⋮,⋮
Zane,TRUE,38.0
Zelda,TRUE,14.0
Zoe,TRUE,16.5
